In [1]:
# Pre-Processing

In [2]:
import pandas as pd
import os

In [3]:
# read in data (TODO: all years)
dat = pd.read_csv(os.path.join("data", "mlb_data_2014.csv"))
dat.head(10)

,Rk,Name,Age,Tm,Lg,G,PA,AB,R,H,...,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Pos Summary
0,1.0,Fernando Abad*\abadfe01,28.0,OAK,AL,7,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0,0,0,1
1,2.0,Bobby Abreu*\abreubo01,40.0,NYM,NL,78,155,133,12,33,...,0.338,0.680,97.0,45,3,0,0,2,0,H9/7D
2,3.0,José Abreu\abreujo02,27.0,CHW,AL,145,622,556,80,176,...,0.581,0.964,173.0,323,14,11,0,4,15,*3D/H
3,4.0,Tony Abreu#\abreuto01,29.0,SFG,NL,3,4,4,0,0,...,0.000,0.000,-100.0,0,1,0,0,0,0,/45H
4,5.0,Alfredo Aceves\aceveal01,31.0,NYY,AL,4,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0,0,0,1
5,6.0,Dustin Ackley*\ackledu01,26.0,SEA,AL,143,542,502,64,123,...,0.398,0.692,99.0,200,10,3,3,2,1,*7H/D
6,7.0,Cristhian Adames#\adamecr01,22.0,COL,NL,7,15,15,1,1,...,0.067,0.133,-65.0,1,1,0,0,0,0,/64H
7,8.0,Lane Adams\adamsla01,24.0,KCR,AL,6,3,3,1,0,...,0.000,0.000,-100.0,0,0,0,0,0,0,/H8D7
8,9.0,Matt Adams*\adamsma01,25.0,STL,NL,142,563,527,55,152,...,0.457,0.779,116.0,241,9,3,0,7,5,*3H
9,10.0,Mike Adams\adamsmi03,35.0,PHI,NL,22,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0,0,0,1


In [4]:
def easy_name(name):
    name = name.split("\\")[0]
    if name[-1] == "#" or name[-1] == "*":
        name = name[:-1]
    return name

def data_etl(dat, year):
    # Remove last row: league averages
    dat = dat.iloc[:-1]
    
    # Remove dupes, add unique ID
    for index, row in dat.iterrows():
        if row["Tm"] == "TOT":
            dat = dat.drop(dat[(dat['Name'] == row["Name"]) & (dat["Tm"] != "TOT")].index) 
        #row["ID"] = row["Name"] + str(year)

    # Exclude pitchers
    dat = dat[dat["Pos Summary"] != "1"]
    dat = dat[dat["Pos Summary"] != "/1"]

    # Exclude players with < 50 plate appearances
    dat = dat[dat["PA"] >= 50]

    # Make names easier to read
    dat['Name'] = dat['Name'].apply(easy_name)

    # Add Year Column, ID Column
    dat["Year"] = year
    dat["ID"] = dat["Name"] + dat["Year"].apply(str)
    return dat


In [5]:
# ETL and concatenate data from all years
dat = pd.read_csv(os.path.join("data", "mlb_data_2014.csv"))
dat = data_etl(dat, 2014)
print(len(dat))
for i in [2015, 2016, 2017, 2018, 2019, 2021]:
    dat_new = pd.read_csv(os.path.join("data", "mlb_data_" + str(i) + ".csv"))
    dat_new = data_etl(dat_new, i)
    dat = pd.concat([dat, dat_new])
    print(len(dat))

522
1044
1567
2092
2616
3165
3708


In [6]:
# Make sure all years are in data
dat[dat["Name"] == "Kolten Wong"]

,Rk,Name,Age,Tm,Lg,G,PA,AB,R,H,...,OPS+,TB,GDP,HBP,SH,SF,IBB,Pos Summary,Year,ID
1379,1380.0,Kolten Wong,23.0,STL,NL,113,433,402,52,100,...,89.0,156,12,4,5,1,3,4H,2014,Kolten Wong2014
1472,1473.0,Kolten Wong,24.0,STL,NL,150,613,557,71,146,...,93.0,215,10,15,0,5,2,*4/H,2015,Kolten Wong2015
1444,1445.0,Kolten Wong,25.0,STL,NL,121,361,313,39,75,...,84.0,111,3,9,0,5,2,4H/879,2016,Kolten Wong2016
1424,1425.0,Kolten Wong,26.0,STL,NL,108,411,354,55,101,...,109.0,146,4,12,1,3,11,4/H,2017,Kolten Wong2017
1510,1511.0,Kolten Wong,27.0,STL,NL,127,407,353,41,88,...,97.0,137,6,14,6,3,3,*4H,2018,Kolten Wong2018
1526,1527.0,Kolten Wong,28.0,STL,NL,148,549,478,61,136,...,108.0,202,2,13,6,5,5,*4/H,2019,Kolten Wong2019
1695,1696.0,Kolten Wong,30.0,MIL,NL,116,492,445,70,121,...,108.0,199,2,13,0,3,1,*4/H,2021,Kolten Wong2021


In [7]:
# Save master data file
dat.to_csv("master_data.csv")